In [ ]:
# 📦 Instalar bibliotecas necessárias
!pip install wordcloud pandas matplotlib


In [ ]:
# 📚 Importação de bibliotecas
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import os


In [ ]:
# 📁 Carregar o arquivo de manifestações
# Suba seu arquivo 'Manifestacoes_Abril2025.xlsx' para o Colab antes de rodar esta célula
arquivo = '/content/Manifestacoes_Abril2025.xlsx'
df = pd.read_excel(arquivo)


In [ ]:
# 🛠️ Ajuste de colunas, caso a primeira linha não seja o cabeçalho real
if df.columns[0] != 'COD_MANIFESTACAO':
    df.columns = df.iloc[0]
    df = df[1:]
    df.columns.name = None


In [ ]:
# 🏷️ Renomear colunas principais
df = df.rename(columns={
    "COD_MANIFESTACAO": "COD_MANIFESTACAO",
    "MOTIVO": "MOTIVO",
    "DATA_MANIF": "DATA_MANIF",
    "DIA_SEM": "DIA_SEM",
    "DATA_OCOR": "DATA_OCOR",
    "HORA_OCOR": "HORA_OCOR",
    "LOCAL_REF": "LOCAL_REF",
    "DESCRIÇÃO": "DESCRICAO",
    "CLASSIFICACAO_MANIF": "CLASSIFICACAO_MANIF",
    df.columns[-1]: "LINHA"
})


In [ ]:
# ⏱️ Converter tipos de dados (datas e horas)
df["HORA_OCOR"] = pd.to_numeric(df["HORA_OCOR"], errors='coerce')
df["DATA_MANIF"] = pd.to_datetime(df["DATA_MANIF"], errors='coerce')
df["DATA_OCOR"] = pd.to_datetime(df["DATA_OCOR"], errors='coerce')


In [ ]:
# 🎯 Filtrar apenas reclamações
df_reclamacoes = df[df["CLASSIFICACAO_MANIF"].str.upper() == "RECLAMAÇÃO"].copy()


In [ ]:
# 📋 Lista simulada de adjetivos negativos usados nas descrições
adjetivos_exemplo = [
    'lento', 'sujo', 'barulhento', 'quente', 'frio', 'perigoso', 'velho', 'lotado', 'demorado',
    'inseguro', 'desconfortável', 'escuro', 'caro', 'úmido', 'horrível', 'ruim', 'péssimo',
    'fraco', 'frágil', 'instável', 'precário', 'ineficiente', 'insuficiente', 'revoltante',
    'desumano', 'inaceitável', 'desagradável', 'injusto', 'problemático', 'difícil', 'caótico',
    'quebrado', 'falho', 'irregular', 'frequente', 'inconstante', 'baixa', 'curto', 'agressivo',
    'confuso', 'degradante', 'frustrante', 'impróprio', 'inadequado', 'insuportável', 'lamentável',
    'mofado', 'obsoleto', 'pobre', 'saturado', 'sofrido', 'sufocante', 'tenso', 'triste', 'vazio',
    'interminável', 'irresponsável', 'inoperante', 'defeituoso', 'danificado', 'parado', 'pior'
]
adjetivos_set = set([a.lower() for a in adjetivos_exemplo if len(a) >= 3])


In [ ]:
# 🧠 Coletar adjetivos encontrados nas reclamações por linha
adjetivos_linha = defaultdict(list)
for linha in df_reclamacoes["LINHA"].unique():
    descricoes = df_reclamacoes[df_reclamacoes["LINHA"] == linha]["DESCRICAO"].dropna().astype(str).tolist()
    palavras = " ".join(descricoes).lower().split()
    palavras_filtradas = [p for p in palavras if p in adjetivos_set]
    adjetivos_linha[linha] = palavras_filtradas


In [ ]:
# ☁️ Gerar nuvem de palavras por linha com base nos adjetivos
output_folder = "/content/nuvens_adjetivos"
os.makedirs(output_folder, exist_ok=True)

for linha, lista_adjetivos in adjetivos_linha.items():
    if len(lista_adjetivos) < 5:
        continue
    texto_final = " ".join(lista_adjetivos)
    wordcloud = WordCloud(
        width=1000,
        height=500,
        background_color='white',
        colormap='inferno',
        max_words=100
    ).generate(texto_final)
    caminho = os.path.join(output_folder, f"{linha.replace(' ', '_')}.png")
    wordcloud.to_file(caminho)


In [ ]:
# 📊 Gerar ranking completo de adjetivos por linha e salvar como Excel
ranking_completo = []
for linha, lista_adjetivos in adjetivos_linha.items():
    contagem = Counter(lista_adjetivos)
    for adjetivo, freq in contagem.items():
        ranking_completo.append({
            "Linha": linha,
            "Adjetivo": adjetivo,
            "Frequência": freq
        })

df_ranking = pd.DataFrame(ranking_completo).sort_values(by=["Linha", "Frequência"], ascending=[True, False])
df_ranking.to_excel("/content/adjetivos_por_linha.xlsx", index=False)

print("✅ Tudo pronto! As nuvens foram salvas na pasta /content/nuvens_adjetivos")
print("📊 O ranking completo está em: /content/adjetivos_por_linha.xlsx")
